In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
from pathlib import Path

In [4]:
datasetpath = Path("./private/data")
filepath = datasetpath/"geometrics_ideal_clean.feather"
data = pd.read_feather(filepath)

In [5]:
data.describe()

,index,endo,epi,volume,longitudinal_distance_epi,longitudinal_distance_endo,longitudinal_distance,radial_distance_epi,radial_distance_endo,radial_distance,...,radial_distance-b,wall_thickness-b,global_longitudinal_length_epi-b,global_longitudinal_length_endo-b,global_longitudinal_length-b,global_circumferential_length_epi-b,global_circumferential_length_endo-b,global_circumferential_length-b,gamma-b,pressure-b
count,677897.000000,677897.000000,677897.000000,677897.000000,677897.000000,677897.000000,677897.000000,677897.000000,677897.000000,677897.000000,...,677897.000000,677897.000000,677897.000000,677897.000000,677897.000000,677897.000000,677897.000000,677897.000000,677897.000000,677897.000000
mean,217.664275,60.247262,-60.234224,0.631149,92.758239,81.325371,87.041805,29.234901,17.255647,23.245274,...,15.366995,35.676287,16.178371,15.586304,15.767860,15.508858,15.320653,15.366172,34.934874,22.806482
std,127.086315,12.101032,12.084862,0.187366,5.599728,5.899347,5.734114,1.039504,2.123234,1.557329,...,8.676628,8.610000,10.095547,9.095807,9.417975,8.888387,8.602388,8.675380,9.671232,14.990435
min,0.000000,40.000000,-80.000000,0.347878,79.814876,68.420181,74.300520,25.958233,12.732181,19.345207,...,1.000000,17.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,107.000000,50.000000,-70.000000,0.461303,88.700477,76.816276,82.770304,28.464258,15.363116,21.918260,...,8.000000,28.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,28.000000,9.000000
50%,217.000000,60.000000,-60.000000,0.601777,92.593784,81.134428,86.868552,29.166966,17.122225,23.134236,...,15.000000,36.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,35.000000,22.000000
75%,327.000000,70.000000,-50.000000,0.788922,96.434445,85.381866,90.883974,30.036527,19.090405,24.549249,...,23.000000,43.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,43.000000,36.000000
max,440.000000,80.000000,-40.000000,0.999999,113.488165,102.992773,108.240468,31.694588,21.607507,26.651047,...,34.000000,50.000000,50.000000,47.000000,49.000000,35.000000,33.000000,34.000000,50.000000,50.000000


In [6]:
assert len(data.columns.to_series()[np.isinf(data).any()]) == 0
assert data.isnull().values.any() == False
assert data.isin([np.inf, -np.inf]).values.any() == False

In [7]:
train, test = train_test_split(data, test_size=0.4, random_state=42) 
valid, test = train_test_split(test, test_size=0.4, random_state=42) 

In [8]:
print("Train: {:.3f}, Validation: {:.3f}, Test:{:.3f}".format(
    len(train)/len(data), len(valid)/len(data), len(test)/len(data) ))

Train: 0.600, Validation: 0.240, Test:0.160


In [9]:
def split_files(df, path, mode, n=100):

  s_space = np.linspace(0,df.count().max(), n, dtype=int, endpoint=True)

  if not os.path.exists(os.path.join(path, mode)):
    os.makedirs(os.path.join(path, mode))

  for i, (iL, iR) in enumerate(zip(s_space[:], s_space[1:])):
    elems = df.iloc[iL:iR]
    elems = elems.sample(frac=1).reset_index(drop=True)
    elems.to_csv("{}/{}/{}.csv".format(path, mode, (int(i))), index=False)

In [10]:
split_files(train, datasetpath, "geometrics_ideal_clean/train")
split_files(valid, datasetpath, "geometrics_ideal_clean/valid")
split_files(test, datasetpath,  "geometrics_ideal_clean/test")